In [1]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('asl_model.h5')

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Start video capture
cap = cv2.VideoCapture(0)  # 0 is the default camera

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # Flip the image horizontally for a selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the BGR image to RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and find hand landmarks
    results = hands.process(rgb)

    # If hand landmarks are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Prepare data for prediction
            landmarks = np.array([[landmark.x, landmark.y] for landmark in hand_landmarks.landmark])
            landmarks = landmarks.flatten()[np.newaxis, ...]

            # Predict gesture
            prediction = model.predict(landmarks)
            class_index = np.argmax(prediction)
            class_label = encoder.inverse_transform([class_index])[0]

            # Display the predicted class label on the image
            cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the image
    cv2.imshow('ASL Gesture Recognition', frame)

    # Exit if ESC key is pressed
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 368ms/step


NameError: name 'encoder' is not defined

: 

In [ ]:
# Predict labels for test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy*100}%")

# Calculate precision, recall, and F1 score
report = classification_report(np.argmax(y_test, axis=1), y_pred, output_dict=True)
print(f"Precision: {report['macro avg']['precision']*100}%")
print(f"Recall: {report['macro avg']['recall']*100}%")
print(f"F1 Score: {report['macro avg']['f1-score']*100}%")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict labels for test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

# Generate confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()